# High-level TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params_lstm import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
Tensorflow:  1.4.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [3]:
EPOCHS = 200
LR = 0.0001
BATCHSIZE = 32
N_CLASSES = 10
N_STEPS = 32
N_INPUTS = 32*3
N_HIDDEN = 64  # RNN units 

In [4]:
def create_symbol(X, n_steps=N_STEPS, nhid=N_HIDDEN, n_classes=N_CLASSES):
    # Convert x to a list[max_time] where element has shape=2 [batch_size, depth]
    x = tf.unstack(X, n_steps, axis=1)
    cell = tf.nn.rnn_cell.BasicRNNCell(nhid)
    # Should really use dynamic_rnn which expects diff input-shape
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    logits = tf.layers.dense(outputs[-1], n_classes, activation=None, name='output')
    return logits

In [5]:
def init_model(m, y, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(lr, b1, b2, eps)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Original data for CNN
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 737 ms, sys: 541 ms, total: 1.28 s
Wall time: 1.28 s


In [7]:
# For RNN
# Sequences of 32 time-steps, each containing 32*3 units
x_train = x_train.reshape(x_train.shape[0], N_STEPS, N_INPUTS)
x_test = x_test.reshape(x_test.shape[0], N_STEPS, N_INPUTS)
print(x_train.shape, x_test.shape)

(50000, 32, 96) (10000, 32, 96)


In [8]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, N_STEPS, N_INPUTS])
y = tf.placeholder(tf.int32, shape=[None])  # Sparse
# Initialise model
sym = create_symbol(X)

CPU times: user 855 ms, sys: 131 ms, total: 986 ms
Wall time: 988 ms


In [9]:
%%time
model = init_model(sym, y)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 698 ms, sys: 751 ms, total: 1.45 s
Wall time: 1.46 s


In [11]:
%%time
for j in range(1,EPOCHS+1):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
        # Accuracy
    if (j%20==0):
        acc_train = sess.run(accuracy, feed_dict={X: x_train, y: y_train})
        print(j, "Train accuracy:", acc_train)

20 Train accuracy: 0.39516
40 Train accuracy: 0.4297
60 Train accuracy: 0.4444
80 Train accuracy: 0.46246
100 Train accuracy: 0.4704
120 Train accuracy: 0.47798
140 Train accuracy: 0.46942
160 Train accuracy: 0.50018
180 Train accuracy: 0.50432
200 Train accuracy: 0.50744
CPU times: user 26min 24s, sys: 2min 31s, total: 28min 55s
Wall time: 27min 12s


In [ ]:
%%time
# Main evaluation loop: 3.22s
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

In [ ]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))